# Linear Optimization Methods

cycle through scipy linprog w/ every combination of layout for max layouts setting. return best answer(s)


In [510]:
import sys
sys.path.append("../")

import numpy as np
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px
import math
from random import shuffle, choice
import copy
from utils import *
from engine import *
from IPython.display import clear_output
import time
import datetime
import re

from scipy.optimize import linprog
from collections import Counter
import itertools

In [512]:
df = load_schedule()

In [513]:
w, q, s, L = process_schedule(df)

The important variables

widths: [160, 195, 205, 220, 235] (mm)
neck in: [4, 5, 5, 7, 7] (mm)
jumbo length (L): 102000 (m)
undeckled jumbos needed (q): [54, 344, 196, 212, 17]


In [514]:
init_layouts(B, w)

[[-25, 0, 0, 0, 0],
 [0, -20, 0, 0, 0],
 [0, 0, -19, 0, 0],
 [0, 0, 0, -18, 0],
 [0, 0, 0, 0, -17]]

In [8]:
obj = [1,1,1,1,1]
lhs_ineq = init_layouts(B, w)
rhs_ineq = [-i for i in q]
print(rhs_ineq)
lhs_ineq

[-54, -344, -196, -212, -17]


[[-25, 0, 0, 0, 0],
 [0, -20, 0, 0, 0],
 [0, 0, -19, 0, 0],
 [0, 0, 0, -18, 0],
 [0, 0, 0, 0, -17]]

In [9]:
linprog(c=obj, 
        A_ub=lhs_ineq,
        b_ub=rhs_ineq, 
        method="revised simplex")

     con: array([], dtype=float64)
     fun: 42.45356725146199
 message: 'The solution was determined in presolve as there are no non-trivial constraints.'
     nit: 0
   slack: array([0., 0., 0., 0., 0.])
  status: 0
 success: True
       x: array([ 2.16      , 17.2       , 10.31578947, 11.77777778,  1.        ])

In [10]:
s = BinPackingExample(w, q)

# Linear Programming

![](https://files.realpython.com/media/lp-py-eq-3.65b2e6d529bc.png)

In [12]:
obj = [-1, -2]
lhs_ineq = [[2, 1],
            [-4, 5],
            [1, -2]]

rhs_ineq = [20, 
            10, 
            2]

lhs_eq = [[-1, 5]]

rhs_eq = [15]

bnd = [(0, float("inf")), #bounds of x
       (0, float("inf"))] # bounds of y

In [13]:
linprog(c=obj, 
        A_ub=lhs_ineq,
        b_ub=rhs_ineq, 
        A_eq=lhs_eq,
        b_eq=rhs_eq,
        bounds=bnd,
        method="revised simplex")

     con: array([0.])
     fun: -16.818181818181817
 message: 'Optimization terminated successfully.'
     nit: 3
   slack: array([ 0.        , 18.18181818,  3.36363636])
  status: 0
 success: True
       x: array([7.72727273, 4.54545455])

## Stock Cutting

![](https://scipbook.readthedocs.io/en/latest/_images/csp-sol-a.png)

In [14]:
s = [6, 6, 5, 5, 5, 4, 4, 4, 4, 2, 2, 2, 2, 3, 3, 7, 7, 5, 5, 8, 8, 4, 4, 5]
w = [2,3,4,5,6,7,8]
B = 9

In [16]:
Counter(s)

Counter({6: 2, 5: 6, 4: 6, 2: 4, 3: 2, 7: 2, 8: 2})

In [17]:
# our layouts 
obj = [1,1,1,1,1,1,1]

# columns are layouts, rows are products
lhs_ineq = [[-4,0,0,0,0,0,0],
            [0,-2,0,0,0,0,0],
            [0,0,-2,0,0,0,0],
            [0,0,0,-1,0,0,0],
            [0,0,0,0,-1,0,0],
            [0,0,0,0,0,-1,0],
            [0,0,0,0,0,0,-1]]

rhs_ineq = [-4, 
            -2, 
            -6,
            -6,
            -2,
            -2, 
            -2]

bnd = [(0, float("inf")), #bounds of x
       (0, float("inf"))] # bounds of y

In [18]:
linprog(c=obj, 
        A_ub=lhs_ineq,
        b_ub=rhs_ineq, 
        method="revised simplex")

     con: array([], dtype=float64)
     fun: 17.0
 message: 'The solution was determined in presolve as there are no non-trivial constraints.'
     nit: 0
   slack: array([0., 0., 0., 0., 0., 0., 0.])
  status: 0
 success: True
       x: array([1., 1., 3., 6., 2., 2., 2.])

## Integer Knapsack Problem

In [515]:
# This is the memoization approach of  
# 0 / 1 Knapsack in Python in simple  
# we can say recursion + memoization = DP 
def knapsack(wt, val, W, n):     
  
    # base conditions 
    if n == 0 or W == 0: 
        return 0
    if t[n][W] != -1: 
        return t[n][W] 
  
    # choice diagram code 
    if wt[n-1] <= W: 
        t[n][W] = max( 
            val[n-1] + knapsack( 
            wt, val, W-wt[n-1], n-1),  
            knapsack(wt, val, W, n-1)) 
        return t[n][W] 
    elif wt[n-1] > W: 
        t[n][W] = knapsack(wt, val, W, n-1) 
        return t[n][W] 

def reconstruct(i, w, kp_soln, weight_of_item):
    """
    Reconstruct subset of items i with weights w. The two inputs
    i and w are taken at the point of optimality in the knapsack soln

    In this case I just assume that i is some number from a range
    0,1,2,...n
    """
    recon = set()
    # assuming our kp soln converged, we stopped at the ith item, so
    # start here and work our way backwards through all the items in
    # the list of kp solns. If an item was deemed optimal by kp, then
    # put it in our bag, otherwise skip it.
    for j in range(i)[::-1]:
        cur_val = kp_soln[j][w]
        prev_val = kp_soln[j-1][w]
        if cur_val > prev_val:
            recon.add(j)
            w = w - weight_of_item[j]
    return recon

def initt(W, n):
    # We initialize the matrix with -1 at first. 
    return [[-1 for i in range(W + 1)] for j in range(n + 1)] 

In [516]:
# driver code 
val = [60, 100, 120] 
wt = [1,1,1] 
W = 2
n = len(val) 
t = initt(W, n)
knapsack(wt, val, W, n)

220

In [21]:
t = np.array(t)
n, w = np.unravel_index(t.argmax(), t.shape) # returns n, W
print(n,w)
sack = reconstruct(n, w, t, wt)
np.array(val)[list(sack)]

3 2


array([100, 120])

In [22]:
s = [6, 6, 5, 5, 5, 4, 4, 4, 4, 2, 2, 2, 2, 3, 3, 7, 7, 5, 5, 8, 8, 4, 4, 5]
B = 9
t = initt(B, len(s))
knapsack(s, s, B, len(s))

9

In [23]:
t = np.array(t)
n, w = np.unravel_index(t.argmax(), t.shape) # returns n, W
print(n,w)

6 9


In [24]:
for pair in np.argwhere(t == t.max()):
    n, w = pair
    sack = reconstruct(n, w, t, s)
    print(sack)

{1}
{3, 6}
{3, 6}
{3, 6}
{3, 6}
{3, 6}
{3, 6}
{3, 6}
{3, 6}
{3, 6}
{3, 6}
{3, 6}
{3, 6}
{3, 6}
{3, 6}
{3, 6}
{3, 6}
{3, 6}
{3, 6}


In [25]:
sack = reconstruct(7, w, t, s)
sack

{3, 6}

In [26]:
np.array(s)[list(sack)]

array([5, 4])

In [27]:
B = 4160
w = list(np.array(widths) + np.array(neckins)) # the values used in the actual computation
q = [math.ceil(x/L) for x in lm] # jumbos needed per width, rounded up
s = BinPackingExample(w, q) # material orders (list of widths, 1 width = 1 jumbo)
t = [[-1 for i in range(B + 1)] for j in range(len(s) + 1)] 

knapsack(s, s, B, len(s))

4160

In [29]:
t = np.array(t)
n, w = np.unravel_index(t.argmax(), t.shape) # returns n, W
print(n,w)

410 4160


In [31]:
sack = reconstruct(n, w, t, s)
Counter(np.array(s)[list(sack)])

Counter({164: 7, 210: 1, 200: 14})

In [32]:
np.array(s)[list(sack)].sum()

4158

In [36]:
for bit in range(0,32*2,2):
    found = 0
    for pair in np.argwhere(t == t.max()-bit):
        n, w = pair
        sack = reconstruct(n, w, t, s)
        print(n,w)
        print(Counter(np.array(s)[list(sack)]))
        print(np.array(s)[list(sack)].sum(), end='\n\n')
        found += 1
        if found > 1:
            break

410 4160
Counter({200: 14, 164: 7, 210: 1})
4158

411 4160
Counter({210: 12, 164: 10})
4160

399 4160
Counter({164: 18, 200: 6})
4152

400 4160
Counter({200: 14, 164: 7, 210: 1})
4158

60 4160
Counter({164: 24, 200: 1})
4136

61 4160
Counter({164: 18, 200: 6})
4152

55 4160
Counter({164: 25})
4100

56 4160
Counter({164: 24, 200: 1})
4136

25 4160
Counter({164: 24})
3936

26 4160
Counter({164: 25})
4100



In [35]:
210*12+164*10

4160

# Knapsack with Cutting Stock

In [321]:
# sumarize inventory as sq m produced

# how to constrain problem so that
# we have max 3 widths per layout
# of 2 layouts
# for 3 layouts

# if we have < 3 unique layouts
# to constrain to 2 layouts
# make the next naive layout pattern the one
# with the greatest amount of sq m inventory

In [676]:
# This is the memoization approach of  
# 0 / 1 Knapsack in Python in simple  
# we can say recursion + memoization = DP 
def knapsack(wt, val, W, n):     
  
    # base conditions 
    if n == 0 or W == 0: 
        return 0
    if t[n][W] != -1: 
        return t[n][W] 
  
    # choice diagram code 
    if wt[n-1] <= W: 
        t[n][W] = max( 
            val[n-1] + knapsack( 
            wt, val, W-wt[n-1], n-1),  
            knapsack(wt, val, W, n-1)) 
        return t[n][W] 
    elif wt[n-1] > W: 
        t[n][W] = knapsack(wt, val, W, n-1) 
        return t[n][W] 

def reconstruct(i, w, kp_soln, weight_of_item):
    """
    Reconstruct subset of items i with weights w. The two inputs
    i and w are taken at the point of optimality in the knapsack soln

    In this case I just assume that i is some number from a range
    0,1,2,...n
    """
    recon = set()
    # assuming our kp soln converged, we stopped at the ith item, so
    # start here and work our way backwards through all the items in
    # the list of kp solns. If an item was deemed optimal by kp, then
    # put it in our bag, otherwise skip it.
    for j in range(i)[::-1]:
        cur_val = kp_soln[j][w]
        prev_val = kp_soln[j-1][w]
        if cur_val > prev_val:
            recon.add(j)
            w = w - weight_of_item[j]
    return recon

def initt(W, n):
    # We initialize the matrix with -1 at first. 
    return [[-1 for i in range(W + 1)] for j in range(n + 1)] 

def make_best_pattern(q, w, n, usable_width=4160):
    """
    Creates the best possible pattern such that all orders are fullfilled in a single
    layout
    
    Parameters
    ----------
    q: list
        rolls required (in jumbo lengths)
    w: list
        widths required
    n: list
        neckins for widths
    usable_width: int
        jumbo/doff usable width
        
    Returns
    -------
    layout: list
        cuts for jumbo for each width (no width is excluded)
    """
    layout = [max(1, math.floor(i/sum(q)*usable_width/j)) for i,j in zip(q,w)] 

    # give priority to widths that had to round down the most
    # when filling up the rest of the pattern
    remainder = [math.remainder(i/sum(q)*usable_width/j, 1) if (math.remainder(i/sum(q)*usable_width/j, 1)
                                                        < 0) else -1 for i,j in zip(q,w) ] 
    order = np.argsort(remainder)
    while (usable_width - sum([i*j for i,j in zip(layout,w)])) > min(w):
        for i in order[::-1]:
            layout[i] += 1
            if usable_width - sum([i*j for i,j in zip(layout,w)]) < 0:
                layout[i] -= 1

    # compute the loss for the final layout
    layout_loss = usable_width - sum([i*j for i,j in zip(layout,w)])
    print("layout pattern: {}".format(dict(zip([i-j for i,j in zip(w,n)],layout))))
    print("pattern loss: {:0.2f} %".format(layout_loss/usable_width*100))

    # multiply to get the minimum doffs required
    # layout * doffs > q
    doffs = max([math.ceil(i/j) for i,j in zip(q, layout)])
    print("minimum doffs to fill order: {}".format(doffs))

    # what inventory is created
    inventory = dict(zip([i-j for i,j in zip(w,n)],[i*doffs-j for i,j in zip(layout,q)]))
    print("inventory created: {}".format(inventory))
    
    return layout

def init_layouts(B, w):
    t = []
    m = len(w)
    for i in range(m):
        pat = [0]*m
        pat[i] = -int(B/w[i])
        t.append(pat)
    return t

def store_patterns(goal=5):
    patterns = []
    bit = 0
    while len(patterns) < goal:
        found = 0
        for pair in np.argwhere(t == t.max()-bit):
            N, W = pair
            sack = reconstruct(N, W, t, s)     
            pattern = Counter(np.array(s)[list(sack)])
            loss = round((B - np.array(s)[list(sack)].sum())/B*100,2)
            patterns.append([pattern, loss])
            if len(patterns) >= goal:
                break
            found += 1
            if found > 1:
                break
        bit += 2
    return patterns

def output_results(result, lhs_ineq, B, w, n):
    sheet = np.sum([(i*j) for i,j in zip(w, np.array(lhs_ineq))],axis=0)#*np.ceil(result['x'])
    inventory = dict(zip([i-j for i,j in zip(w,n)],np.sum(np.array(lhs_ineq)*-1*np.ceil(result['x']),axis=1)-np.array(q)))

    # create layout summary
    jumbos = list(np.ceil(result['x'])[np.ceil(result['x'])>0])
    temp = np.array(lhs_ineq)*-1*np.where(np.ceil(result['x']) != 0, 1, 0)
    temp = temp[:, temp.any(0)].T
    non_zero_layouts = list([dict(zip([i-j for i,j in zip(w,n)], i)) for i in temp])

    sheet_loss = [B+i for i in sheet]
    sheet_loss = [i / B * 100 for i,j in zip(sheet_loss,np.where(result['x'] > 0, 1, 0)) if j > 0]

    # remove extra layouts due to ceiling rounding from linprog
    sorted_jumbos = [x for _,x in sorted(zip(sheet_loss,jumbos))][::-1]
    sorted_layouts = np.array(non_zero_layouts)[np.array(sheet_loss).argsort()][::-1]
    sorted_losses = [x for _,x in sorted(zip(sheet_loss,sheet_loss))][::-1]
    for index, layout2 in enumerate(np.array(non_zero_layouts)[np.array(sheet_loss).argsort()][::-1]):
        if all(np.array(list(inventory.values())) - np.array(list(layout2.values())) > 0):
            sorted_jumbos[index] -= 1
            new_values = np.array(list(inventory.values())) - np.array(list(layout2.values()))
            inventory.update(zip(inventory,new_values))

            # clear layouts that have been set to 0
    summary = (list(zip(sorted_jumbos, sorted_layouts, sorted_losses)))
    summ = []
    for i in summary:
        if i[0] > 0:
            summ.append(i)
    summary=summ
    loss = sum([i[0]*i[2] for i in summary])/sum([i[0] for i in summary])

    print("total loss: {:0.2f} %".format(loss), end = '\n\n')
    print("inventory created: {}".format(inventory), end = '\n\n')
    print("total inventory rolls: {:n}".format(sum(list(inventory.values()))), end='\n\n')
    print("layout summary:", end = '\n\n')
    for i in summary:
        print("loss: {:.2f}% \t {} x\t {}".format(i[2], i[0], i[1]))
    print('')
    print("total layouts: {}".format(np.sum(sorted_jumbos)))
    
    return loss, inventory, summary, sorted_jumbos

In [536]:
df = load_schedule(
    customer='AHP',
    technology='SM',
    color='WHITE',
    cycle='CYCLE 1',
)
# df = df.iloc[:20]
B=4160
w, q, L, n = process_schedule(
    df,
    B=B,
    put_up=17000,
    doffs_in_jumbo=6,
    verbiose=True,
)

The important variables

widths: [160, 195, 205, 220, 235] (mm)
neck in: [4, 5, 5, 7, 7] (mm)
jumbo length (L): 102000 (m)
undeckled jumbos needed (q): [54, 344, 196, 212, 17]


In [562]:
layout = make_best_pattern(q, w, n)

layout pattern: {160: 2, 195: 8, 205: 4, 220: 5, 235: 1}
pattern loss: 0.36 %
minimum doffs to fill order: 49
inventory created: {160: 44, 195: 48, 205: 0, 220: 33, 235: 32}


In [663]:
# choose max unique widths per doff
max_combinations = 3
combos = list(itertools.combinations(w,r=max_combinations))
print("{} possible max {} combinations".format(len(combos),max_combinations))
patterns = []
for combo in combos:
    # only provide knapsack with relevant variables
    s = []
    for i in combo:
        s += (int(B/i)*[i])
        
    t = initt(B,len(s))
    knapsack(s, s, B, len(s))
    t = np.array(t)
    patterns += store_patterns(goal=5)

10 possible max 3 combinations


In [664]:
uni_list = []
for i in patterns:
    if i not in uni_list:
        uni_list.append(i)
patterns = uni_list
print("{} unique patterns found".format(len(patterns)))

30 unique patterns found


In [670]:
# find best of X combination
max_patterns = 2
pattern_combos = list(itertools.combinations(patterns,r=max_patterns))
print("{} possible max {} patterns".format(len(pattern_combos),max_patterns))

435 possible max 2 patterns


In [671]:
best_of = []
for combo in pattern_combos:
    patterns2 = combo
    lhs_ineq = []
    for pattern in patterns2:
        inset = []
        for width in w:
            try:
                inset.append(-pattern[0][width])
            except:
                inset.append(0)
        lhs_ineq.append(inset)
#     naive = init_layouts(B, w)
#     lhs_ineq = lhs_ineq + naive

    lhs_ineq.append([-i for i in layout])
    lhs_ineq = np.array(lhs_ineq).T.tolist()
    rhs_ineq = [-i for i in q]
    obj = np.ones(len(lhs_ineq[0]))

    result = linprog(c=obj, 
            A_ub=lhs_ineq,
            b_ub=rhs_ineq, 
            method="revised simplex")
    
    sheet = np.sum([(i*j) for i,j in zip(w, np.array(lhs_ineq))],axis=0)#*np.ceil(result['x'])
    inventory = dict(zip([i-j for i,j in zip(w,n)],np.sum(np.array(lhs_ineq)*-1*\
                                                          np.ceil(result['x']),axis=1)-np.array(q)))

    # create layout summary
    jumbos = list(np.ceil(result['x'])[np.ceil(result['x'])>0])
    temp = np.array(lhs_ineq)*-1*np.where(np.ceil(result['x']) != 0, 1, 0)
    temp = temp[:, temp.any(0)].T
    non_zero_layouts = list([dict(zip([i-j for i,j in zip(w,n)], i)) for i in temp])

    sheet_loss = [B+i for i in sheet]
    sheet_loss = [i / B * 100 for i,j in zip(sheet_loss,np.where(result['x'] > 0, 1, 0)) if j > 0]

    # remove extra layouts due to ceiling rounding from linprog
    sorted_jumbos = [x for _,x in sorted(zip(sheet_loss,jumbos))][::-1]
    sorted_layouts = np.array(non_zero_layouts)[np.array(sheet_loss).argsort()][::-1]
    sorted_losses = [x for _,x in sorted(zip(sheet_loss,sheet_loss))][::-1]
    for index, layout2 in enumerate(np.array(non_zero_layouts)[np.array(sheet_loss).argsort()][::-1]):
        if all(np.array(list(inventory.values())) - np.array(list(layout2.values())) > 0):
            sorted_jumbos[index] -= 1
            new_values = np.array(list(inventory.values())) - np.array(list(layout2.values()))
            inventory.update(zip(inventory,new_values))

            # clear layouts that have been set to 0
    summary = (list(zip(sorted_jumbos, sorted_layouts, sorted_losses)))
    summ = []
    for i in summary:
        if i[0] > 0:
            summ.append(i)
    summary=summ
    loss = sum([i[0]*i[2] for i in summary])/sum([i[0] for i in summary])
    
    best_of.append([loss, sum(list(inventory.values())), patterns2])

In [674]:
# minimize inventory or minimize mat. loss
inv_loss = 0
arr = np.array(best_of)
print(arr[np.argmin(arr[:,inv_loss])])
patterns_final = arr[np.argmin(arr[:,inv_loss])][2]

[0.1567725752508361 114.0
 ([Counter({200: 14, 164: 7, 210: 1}), 0.05], [Counter({227: 10, 210: 9}), 0.0])]


In [675]:
# find overall best combination
# format layouts for linear optimization
lhs_ineq = []
for pattern in patterns_final:
    inset = []
    for width in w:
        try:
            inset.append(-pattern[0][width])
        except:
            inset.append(0)
    lhs_ineq.append(inset)
# naive = init_layouts(B, w)
# lhs_ineq = lhs_ineq + naive

lhs_ineq.append([-i for i in layout])
lhs_ineq = np.array(lhs_ineq).T.tolist()
rhs_ineq = [-i for i in q]
obj = np.ones(len(lhs_ineq[0]))

result = linprog(c=obj, 
        A_ub=lhs_ineq,
        b_ub=rhs_ineq, 
        method="revised simplex")

output_results(result, lhs_ineq, B, w, n)

total loss: 0.16 %

inventory created: {160: 87.0, 195: 10.0, 205: 8.0, 220: 8.0, 235: 1.0}

total inventory rolls: 114

layout summary:

loss: 0.36% 	 18.0 x	 {160: 2, 195: 8, 205: 4, 220: 5, 235: 1}
loss: 0.05% 	 15.0 x	 {160: 7, 195: 14, 205: 1, 220: 0, 235: 0}
loss: 0.00% 	 13.0 x	 {160: 0, 195: 0, 205: 9, 220: 10, 235: 0}

total layouts: 46.0


(0.1567725752508361,
 {160: 87.0, 195: 10.0, 205: 8.0, 220: 8.0, 235: 1.0},
 [(18.0, {160: 2, 195: 8, 205: 4, 220: 5, 235: 1}, 0.3605769230769231),
  (15.0, {160: 7, 195: 14, 205: 1, 220: 0, 235: 0}, 0.04807692307692308),
  (13.0, {160: 0, 195: 0, 205: 9, 220: 10, 235: 0}, 0.0)],
 [18.0, 15.0, 13.0])

In [569]:
# for 2 optimum layouts just add the naive layout
naive = init_layouts(B, w)
lhs_ineq = naive

lhs_ineq.append([-i for i in layout])
lhs_ineq = np.array(lhs_ineq).T.tolist()
rhs_ineq = [-i for i in q]
obj = np.ones(len(lhs_ineq[0]))

result = linprog(c=obj, 
        A_ub=lhs_ineq,
        b_ub=rhs_ineq, 
        method="revised simplex")

In [570]:
sheet = np.sum([(i*j) for i,j in zip(w, np.array(lhs_ineq))],axis=0)#*np.ceil(result['x'])
inventory = dict(zip([i-j for i,j in zip(w,n)],np.sum(np.array(lhs_ineq)*-1*np.ceil(result['x']),axis=1)-np.array(q)))

# create layout summary
jumbos = list(np.ceil(result['x'])[np.ceil(result['x'])>0])
temp = np.array(lhs_ineq)*-1*np.where(np.ceil(result['x']) != 0, 1, 0)
temp = temp[:, temp.any(0)].T
non_zero_layouts = list([dict(zip([i-j for i,j in zip(w,n)], i)) for i in temp])

sheet_loss = [B+i for i in sheet]
sheet_loss = [i / B * 100 for i,j in zip(sheet_loss,np.where(result['x'] > 0, 1, 0)) if j > 0]

# remove extra layouts due to ceiling rounding from linprog
sorted_jumbos = [x for _,x in sorted(zip(sheet_loss,jumbos))][::-1]
sorted_layouts = np.array(non_zero_layouts)[np.array(sheet_loss).argsort()][::-1]
sorted_losses = [x for _,x in sorted(zip(sheet_loss,sheet_loss))][::-1]
for index, layout2 in enumerate(np.array(non_zero_layouts)[np.array(sheet_loss).argsort()][::-1]):
    if all(np.array(list(inventory.values())) - np.array(list(layout2.values())) > 0):
        sorted_jumbos[index] -= 1
        new_values = np.array(list(inventory.values())) - np.array(list(layout2.values()))
        inventory.update(zip(inventory,new_values))

        # clear layouts that have been set to 0
summary = (list(zip(sorted_jumbos, sorted_layouts, sorted_losses)))
summ = []
for i in summary:
    if i[0] > 0:
        summ.append(i)
summary=summ
loss = sum([i[0]*i[2] for i in summary])/sum([i[0] for i in summary])

print("total loss: {:0.2f} %".format(loss), end = '\n\n')
print("inventory created: {}".format(inventory), end = '\n\n')
print("total inventory rolls: {:n}".format(sum(list(inventory.values()))), end='\n\n')
print("layout summary:", end = '\n\n')
for i in summary:
    print("loss: {:.2f}% \t {} x\t {}".format(i[2], i[0], i[1]))
print('')
print("total layouts: {}".format(np.sum(sorted_jumbos)))

total loss: 2.11 %

inventory created: {160: 5.0, 195: 12.0, 205: 5.0, 220: 17.0, 235: 0.0}

total inventory rolls: 39

layout summary:

loss: 4.09% 	 7.0 x	 {160: 0, 195: 0, 205: 19, 220: 0, 235: 0}
loss: 3.85% 	 11.0 x	 {160: 0, 195: 20, 205: 0, 220: 0, 235: 0}
loss: 1.78% 	 8.0 x	 {160: 0, 195: 0, 205: 0, 220: 18, 235: 0}
loss: 1.44% 	 1.0 x	 {160: 25, 195: 0, 205: 0, 220: 0, 235: 0}
loss: 0.36% 	 17.0 x	 {160: 2, 195: 8, 205: 4, 220: 5, 235: 1}

total layouts: 44.0
